In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Data loading
from sklearn.datasets import load_iris

# 기계학습 모델 생성, 학습, 평가
import statsmodels.api as sm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("../input/dataset/UniversalBank.csv")
data

### 데이터 변수 확인

In [ ]:
data.columns

## Universal bank 데이터
#### 설명변수 및 반응변수
- ID - 고객 고유번호 <br>
- Age - 고객의 연령 (연속형) <br>
- Experienc - 고객의 경력 (연속형) <br>
- Incom - 고객의 연간 수입 (연속형) <br>
- ZIP Code - 고객 주소 ZIP Code <br>
- Family - 고객의 가족 규모 (연속형) <br>
- CCAvg - 평균 신용 카드 지출 (연속형) <br>
- Education - 교육 수준 (범주형 - 1: 대학교 졸업, 2: 대학원 졸업, 3: 고급/전문) <br>
- Mortgag - 주택 모기지의 가치 (연속형) <br>
- SecuritiesAccount - 고객은 은행에 증권 계좌를 가지고 있는가? (범주형 - 0: 아니오, 1: 예) <br>
- CDAccount - 고객은 은행에 예금 계좌를 가지고 있는가?	(범주형 - 0: 아니오, 1: 예) <br>
- Online - 고객이 인터넷 뱅킹 시설을 사용하는가? (범주형 - 0: 아니오, 1: 예) <br>
- CreditCard - 고객이 Universal Bank에서 발행 한 신용 카드를 사용하는가? (범주형 - 0: 아니오, 1: 예)
- <span style="color:blue">PersonalLoan - 고객은 마지막 캠페인에 제공된 개인 대출을 수락 하였나? (범주형 - 0: 아니오, 1: 예) <b>[타겟 변수]</b> <br></span>

### 불필요 데이터 제거 

In [ ]:
# 불필요 데이터 제거 
# dataFrame_name.drop('column name', axis = 1) 사용
# 여러개의 column을 삭제하고 싶을 때는 dataFrame_name.drop(['column name1','column name2'], axis = 1)
# 사용하고자 하는 변수에 저장해주어야 함 (ex: modified_data = dataFrame_name.drop('column name', axis = 1))


In [ ]:
data = data.drop(['ID','ZIP Code'], axis=1)

### 입력변수 = X, 종속변수 = y 로 구분

In [ ]:
# .drop()을 응용하여 X, y 생성
X = data.drop('Personal Loan', axis=1)
y = data['Personal Loan']

In [ ]:
plt.figure(figsize=(4, 5))
sns.countplot(y)
plt.show()

## 1. 학습/ 테스트 셋 구성

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## 2. Decision Tree model 생성 및 학습

In [ ]:
clf = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 100, max_depth = 2, random_state = 123)
## criterion{“gini”, “entropy”}, default=”gini”

clf.fit(X_train, y_train)

### DecisionTreeClassifier(*, criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, ccp_alpha=0.0)

## 3. Test X에 대한 y 예측

In [ ]:
# 예측 code 입력,y_pred 변수에 정의(model-name.predict(new_X data)사용)


In [ ]:
y_pred =  clf.predict(X_test)

In [ ]:
y_pred

In [ ]:
print('클래스별 데이터 개수: Test')
print(y_test.value_counts())

## 4. 결과 확인

In [ ]:
from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix\n')
print(confusion)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_test, y_pred))

Accuracy = 0.96, 틀린 것 대비 맞춘 비율이 높아 비교적 높은 정확도를 보여주고 있음

## 5. 성능 향상을 위한 hyper-parameter 탐색

- criterion{“gini”, “entropy”}, default=”gini”
- splitter{“best”, “random”}, default=”best”
- max_depthint, default=None
- min_samples_splitint or float, default=2

### 5-1. criterion = Entropy or Gini index

분기 후의 각 노드의 무질서 정도(homogeneity)가 증가하거나 불순도(impurity) or 불확실성(uncertainty)이 최대한 감소하도록 하는 방향으로 학습을 진행하도록 결정<br>
- Entropy는 0에 가까울 때 순도100%이고 엔트로피가 높을 수록 데이터 calss가 분리되지 않음
- Entropy는 순도를 최대화 하는 방향으로 학습

### Entropy

In [ ]:
group_1 = np.array([0.3, 0.4, 0.3])
group_2 = np.array([0.7, 0.2, 0.1])
group_3 = np.array([0.01, 0.01, 0.98])

In [ ]:
fig, axes = plt.subplots(1, 3)
fig.set_size_inches(12, 4)
axes[0].bar(np.arange(3), group_1, color='blue')
axes[0].set_title('Group 1')
axes[1].bar(np.arange(3), group_2, color='red')
axes[1].set_title('Group 2')
axes[2].bar(np.arange(3), group_3, color='green')
axes[2].set_title('Group 3')
plt.show()

In [ ]:
from IPython.display import Image
Image(url='https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png', width=350)

In [ ]:
## entropy 구현
def entropy(x):
    return (-x*np.log2(x)).sum()

In [ ]:
entropy_1 = entropy(group_1)
entropy_2 = entropy(group_2)
entropy_3 = entropy(group_3)

print(f'Group 1: {entropy_1:.3f}\nGroup 2: {entropy_2:.3f}\nGroup 3: {entropy_3:.3f}')

In [ ]:
plt.figure(figsize=(5, 5))
plt.bar(['Group 1', 'Group 2', 'Group 3'], [entropy_1, entropy_2, entropy_3])
plt.title('Entropy', fontsize=15)
plt.show()

### Gini index

- gini도 Entropy와 마찬가지로 불순도(Impurity)를 최소화 하는 방향으로 학습

In [ ]:
Image('../input/lgkoreauniv/gini.JPG', width=350)

In [ ]:
# Gini Index 구현
def gini(x):
    return 1 - ((x / x.sum())**2).sum()

In [ ]:
group_1 = np.array([50, 50])
group_2 = np.array([30, 70])
group_3 = np.array([0, 100])

In [ ]:
fig, axes = plt.subplots(1, 3)
fig.set_size_inches(12, 4)
axes[0].bar(['Positive', 'Negative'], group_1, color='blue')
axes[0].set_title('Group 1')
axes[1].bar(['Positive', 'Negative'], group_2, color='red')
axes[1].set_title('Group 2')
axes[2].bar(['Positive', 'Negative'], group_3, color='green')
axes[2].set_title('Group 3')
plt.show()

In [ ]:
gini_1 = gini(group_1)
gini_2 = gini(group_2)
gini_3 = gini(group_3)

print(f'Group 1: {gini_1:.3f}\nGroup 2: {gini_2:.3f}\nGroup 3: {gini_3:.3f}')

In [ ]:
plt.figure(figsize=(5, 5))
plt.bar(['Group 1', 'Group 2', 'Group 3'], [gini_1, gini_2, gini_3])
plt.title('Gini Index', fontsize=15)
plt.show()

## 기존 학습모델(clf)의 criterion을 entropy로 바꾸어 학습 결과 확인/ 비교

### <b> clf = DecisionTreeClassifier(criterion='gini', min_samples_split=100, random_state = 123, max_depth = 2)</b>

In [ ]:
# criterion 조정
clf_etp = DecisionTreeClassifier(min_samples_split=100, random_state = 123, max_depth = 2)
clf_etp.fit(X_train, y_train)

In [ ]:
y_pred_etp = clf_etp.predict(X_test)
print(classification_report(y_test, y_pred_etp))

### 5-2. min_samples_split 조정 (노드에서 분기 가능토록하는 최소 데이터 수)

In [ ]:
Image("../input/lgkoreauniv/dt1.JPG")

#### <b>clf = DecisionTreeClassifier(criterion='gini', min_samples_split=100, random_state = 123, max_depth = 2)</b>

In [ ]:
clf_msp = DecisionTreeClassifier(criterion='gini', random_state = 123, max_depth = 2)
clf_msp.fit(X_train, y_train)

In [ ]:
y_pred_msp = clf_msp.predict(X_test)
print(classification_report(y_test, y_pred_msp))

너무 작은 min_samples_split은 과적합을 발생시킴

### 5-3. max_depth (Tree의 최대의 깊이 )

- 트리의 최대 깊이를 규정<br>
- 디폴트는 None, 완벽하게 클래스 결정 값이 될 때까지 깊이를 계속 키우며 분할하거나 노드가 가지는 데이터 개수가 min_samples_split 보다 작아질 때까지 계속 깊이를 증가시킴<br>
- (설정한 min_samples_split에 따라 학습되더라도 학습모델의 과적합 결과를 얻을 수 있음, 따라서 max_depth의 제어도 함께 필요함)
- 깊이가 깊어지면 min_sample_split 설정대로 최대 분할하여 과적합할 수 있으므로 적절한 값으로 제어 필요

#### clf = DecisionTreeClassifier(criterion='gini', min_samples_split=100, random_state = 123, max_depth = 2)

In [ ]:
clf_md = DecisionTreeClassifier(criterion='gini', min_samples_split=100, random_state = 123)
clf_md.fit(X_train, y_train)

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(20, 15))
plot_tree(decision_tree=clf, filled=True)
# filled = indicate majority class for classification
plt.show()

fig.savefig('imagename.png')

In [ ]:
plt.figure(figsize=(20, 15))
plot_tree(decision_tree=clf_md, filled=True)
# filled = indicate majority class for classification
plt.show()

fig.savefig('imagename.png')

In [ ]:
y_pred_md = clf_md.predict(X_test)
print(classification_report(y_test, y_pred_md))

##### Tree가 깊어질수록 학습 품질이 더 좋아지지만, 학습 데이터에 과적합되어 유사 데이터인 테스트 셋에서도 좋은 성능을 보일 수 있음<br>
##### 새로운 은행 고객의 데이터 셋을 테스트 할 경우 성능이 현저히 떨어질 수 있음  

## 6. Best Hyper-parameter 탐색

- hyper-parameter들 간에도 성능에 영향을 미칠 수 있음
- 여러가지 parameter 조합 가능
- 수동 조정을 통해 최적 성능을 갖을 수 있는지 알 수 없으므로 GridsearchCV 모듈을 사용하여 best-patameter 탐색

In [ ]:
from sklearn.model_selection import GridSearchCV
# GridSearchCV의 param_grid 설정

params = {
    "criterion" : ['gini', 'entropy'],
    "max_depth" : [3, 4, 5],
    "min_samples_split" : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
}

In [ ]:
clf_best = DecisionTreeClassifier(random_state = 123)

grid_tree = GridSearchCV(clf_best, param_grid=params, cv=10, refit=True)
grid_tree.fit(X_train, y_train)

print('best parameters : ', grid_tree.best_params_)
print('best score : ', grid_tree.best_score_)

best score: 훈련 세트에서 수행한 교차검증의 평균 정확도

In [ ]:
# best_tree_model = DecisionTreeClassifier() Best parameter로 학습 및 X_train, y_train fitting
# y_pred_best = .predict(X_test)
# print(classification_report()) 이용해서 best-parameter로 학습 후 y를 예측했을 때의 성능 table 도출
best_tree_model = grid_tree.best_estimator_

In [ ]:
X_new = np.array([[36, 11, 158, 2, 7.8, 1, 114, 0, 0, 1, 1]])
y_new = 0

In [ ]:
# X_new data로 예측 후, 예측값을 y_pred_new에 저장

In [ ]:
feature_importance_loc = best_tree_model.feature_importances_
df_loc = pd.DataFrame({'feature':X_train.columns,
                       'importance':feature_importance_loc})\
    .sort_values('importance',ascending=False)
#df_scale = pd.DataFrame({'feature':load_boston()['feature_names'],
#                       'importance':feature_importance_scale})\
#    .sort_values('importance',ascending=False)

#plt.rc('axes', titlesize=SMALL_SIZE) # fontsize of the axes title
plt.rc('axes', labelsize=15) # fontsize of the x and y labels
#plt.rc('xtick', labelsize=SMALL_SIZE) # fontsize of the tick labels
plt.rc('ytick', labelsize=11) # fontsize of the tick labels
#plt.rc('legend', fontsize=SMALL_SIZE) # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)
plt.rcParams['axes.grid'] = True 
plt.rcParams["figure.figsize"] = (8, 10)
plt.rc('font', size=10)
#plt.title("Feature importance plot for distribution parameters", fontsize=10)
sns.barplot(x='importance',y='feature',data=df_loc[0:30], color="skyblue").set_title('Feature importance plot for distribution parameters')